# Capstone Project - The Battle of Neighborhoods (Week 1)

In [ ]:
Topic: Finding the best restaurant to eat in Oporto
Date: 07 March 2020
Author: Pedro Correia de Sousa
1. Description of the Problem
Oporto is a popular tourist destination for good food where diverse tourists from all around the world gather to eat the famous "francesinha".

According to the data published by the Higher Education Statistical Agency (HESA), in the academic year 2016-2017 London welcomed 112,200 international students to its higher education institutions, which make up 29 percent of students at higher education institutions.

One of the big concerns for international students when moving to a new city would be finding an accommodation. Student halls are the most reliable means of housing for students, but it is not easy to secure a place in one as they are in high demand. Therefore in this project, I intend to explore different neighborhoods of London and find the best area to build a new student hall for international students in London to solve this persistent problem and to find a new business opportunity.This research is expected to benefit real-estate investors looking for a profitable location or international students looking for a place to live in London.

From the student perspective, a lot of factors come into play when finding the best accommodation, including location and rent. Howerver, this study will focus only on the safety and the general atmosphere of the neighborhood. Distance to universities are also an important factor in choosing a student hall, but as student halls accept students from different universities, it will be disregarded in this project.

2. Data Acquisition and Preprocessing
In this project, I will be using the following datasets to help solve my problem - London Recorded Crime, List of London Boroughs, and Foursquare API.

In [3]:
import pandas as pd
import numpy as np

## Week 1 assignment

Print the following the statement: Hello Capstone Project Course!

In [4]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# 1. Extract data of Toronto neighborhoods

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# download url data from internet
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = BeautifulSoup(source, 'lxml')

In [3]:
# creat a new Dataframe
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

In [8]:
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Cleaning the data

In [9]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)

SyntaxError: unexpected EOF while parsing (<ipython-input-9-c13d92e268cf>, line 3)

In [10]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [12]:
print(df2.shape)
df2.head(10)

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [13]:
print('The DataFrame shape is', df2.shape)

The DataFrame shape is (103, 3)


# 2. Add the latitude and longitude coordinates to the dataframe

In [14]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
# download url data from internet
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = BeautifulSoup(source, 'lxml')

# creat a new Dataframe
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')
df2.rename(columns={'Postalcode':'Postal Code'}, inplace=True)
df2.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Retrive postcode coordinates

In [15]:
toronto_geocsv = 'https://cocl.us/Geospatial_data'
!wget -q -O 'toronto_m.geospatial_data.csv' toronto_geocsv
geocsv_data = pd.read_csv(toronto_geocsv).set_index("Postal Code")
geocsv_data.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [16]:
df = pd.merge(geocsv_data, df2, on='Postal Code')
df.head()

,Postal Code,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [53]:
df_2 = df[['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df_2.head(11)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [65]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
Latitude = location.latitude
Longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(Latitude, Longitude))

# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[Latitude, Longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_2['Latitude'], df_2['Longitude'], df_2['Borough'], df_2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

The geograpical coordinate of Toronto are 43.653963, -79.387207.


# 3. Clustering 

In [57]:

df_2.groupby('Borough').count()

,Postal Code,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,11,11,11,11
Mississauga,1,1,1,1
North York,24,24,24,24
Queen's Park,1,1,1,1
Scarborough,17,17,17,17


In [58]:
toronto_onehot = pd.get_dummies(df[['Neighborhood']], prefix="", prefix_sep="")

#add encoded neighborhood to df
toronto_onehot['Borough'] = df['Borough'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Borough,"Adelaide, King, Richmond",Agincourt,"Agincourt North, L'Amoreaux East, Milliken, Steeles East","Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown","Alderwood, Long Branch","Bathurst Manor, Downsview North, Wilson Heights",Bayview Village,"Bedford Park, Lawrence Manor East",Berczy Park,"Birch Cliff, Cliffside West","Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe","Brockton, Exhibition Place, Parkdale Village",Business Reply Mail Processing Centre 969 Eastern,"CFB Toronto, Downsview East","CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara","Cabbagetown, St. James Town",Caledonia-Fairbanks,Canada Post Gateway Processing Centre,Cedarbrae,Central Bay Street,"Chinatown, Grange Park, Kensington Market",Christie,Church and Wellesley,"Clairlea, Golden Mile, Oakridge","Clarks Corners, Sullivan, Tam O'Shanter","Cliffcrest, Cliffside, Scarborough Village West","Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park","Commerce Court, Victoria Hotel",Davisville,Davisville North,"Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West","Del Ray, Keelesdale, Mount Dennis, Silverthorn","Design Exchange, Toronto Dominion Centre",Don Mills North,"Dorset Park, Scarborough Town Centre, Wexford Heights","Dovercourt Village, Dufferin",Downsview Central,Downsview Northwest,Downsview West,"Downsview, North Park, Upwood Park","East Birchmount Park, Ionview, Kennedy Park",East Toronto,"Emery, Humberlea","Fairview, Henry Farm, Oriole","First Canadian Place, Underground city","Flemingdon Park, Don Mills South","Forest Hill North, Forest Hill West",Glencairn,"Guildwood, Morningside, West Hill","Harbord, University of Toronto",Harbourfront,"Harbourfront East, Toronto Islands, Union Station","High Park, The Junction South","Highland Creek, Rouge Hill, Port Union",Hillcrest Village,"Humber Bay Shores, Mimico South, New Toronto","Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea",Humber Summit,Humewood-Cedarvale,"Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips","Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor",L'Amoreaux West,"Lawrence Heights, Lawrence Manor",Lawrence Park,Leaside,"Little Portugal, Trinity","Maryvale, Wexford","Moore Park, Summerhill East","Newtonbrook, Willowdale",North Toronto West,Northwest,"Northwood Park, York University","Parkdale, Roncesvalles",Parkwoods,Queen's Park,Rosedale,Roselawn,"Rouge, Malvern","Runnymede, Swansea","Ryerson, Garden District",Scarborough Village,"Silver Hills, York Mills",St. James Town,Stn A PO Boxes 25 The Esplanade,Studio District,"The Annex, North Midtown, Yorkville",The Beaches,"The Beaches West, India Bazaar","The Danforth West, Riverdale","The Junction North, Runnymede","The Kingsway, Montgomery Road, Old Mill North",Thorncliffe Park,Upper Rouge,Victoria Village,Westmount,Weston,Willowdale South,Willowdale West,Woburn,"Woodbine Gardens, Parkview Hill",Woodbine Heights,York Mills West
0,Scarborough,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Scarborough,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Scarborough,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Scarborough,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [47]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped.head()

,Borough,"Adelaide, King, Richmond",Agincourt,"Agincourt North, L'Amoreaux East, Milliken, Steeles East","Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown","Alderwood, Long Branch","Bathurst Manor, Downsview North, Wilson Heights",Bayview Village,"Bedford Park, Lawrence Manor East",Berczy Park,"Birch Cliff, Cliffside West","Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe","Brockton, Exhibition Place, Parkdale Village",Business Reply Mail Processing Centre 969 Eastern,"CFB Toronto, Downsview East","CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara","Cabbagetown, St. James Town",Caledonia-Fairbanks,Canada Post Gateway Processing Centre,Cedarbrae,Central Bay Street,"Chinatown, Grange Park, Kensington Market",Christie,Church and Wellesley,"Clairlea, Golden Mile, Oakridge","Clarks Corners, Sullivan, Tam O'Shanter","Cliffcrest, Cliffside, Scarborough Village West","Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park","Commerce Court, Victoria Hotel",Davisville,Davisville North,"Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West","Del Ray, Keelesdale, Mount Dennis, Silverthorn","Design Exchange, Toronto Dominion Centre",Don Mills North,"Dorset Park, Scarborough Town Centre, Wexford Heights","Dovercourt Village, Dufferin",Downsview Central,Downsview Northwest,Downsview West,"Downsview, North Park, Upwood Park","East Birchmount Park, Ionview, Kennedy Park",East Toronto,"Emery, Humberlea","Fairview, Henry Farm, Oriole","First Canadian Place, Underground city","Flemingdon Park, Don Mills South","Forest Hill North, Forest Hill West",Glencairn,"Guildwood, Morningside, West Hill","Harbord, University of Toronto",Harbourfront,"Harbourfront East, Toronto Islands, Union Station","High Park, The Junction South","Highland Creek, Rouge Hill, Port Union",Hillcrest Village,"Humber Bay Shores, Mimico South, New Toronto","Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea",Humber Summit,Humewood-Cedarvale,"Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips","Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor",L'Amoreaux West,"Lawrence Heights, Lawrence Manor",Lawrence Park,Leaside,"Little Portugal, Trinity","Maryvale, Wexford","Moore Park, Summerhill East","Newtonbrook, Willowdale",North Toronto West,Northwest,"Northwood Park, York University","Parkdale, Roncesvalles",Parkwoods,Queen's Park,Rosedale,Roselawn,"Rouge, Malvern","Runnymede, Swansea","Ryerson, Garden District",Scarborough Village,"Silver Hills, York Mills",St. James Town,Stn A PO Boxes 25 The Esplanade,Studio District,"The Annex, North Midtown, Yorkville",The Beaches,"The Beaches West, India Bazaar","The Danforth West, Riverdale","The Junction North, Runnymede","The Kingsway, Montgomery Road, Old Mill North",Thorncliffe Park,Upper Rouge,Victoria Village,Westmount,Weston,Willowdale South,Willowdale West,Woburn,"Woodbine Gardens, Parkview Hill",Woodbine Heights,York Mills West
0,Central Toronto,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.111111,0.111111,0.111111,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.111111,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.111111,0.0,0.0,0.0,0.111111,0.0,0.111111,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.111111,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.111111,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Downtown Toronto,0.052632,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.052632,0.0,0.000000,0.0,0.0,0.0,0.052632,0.052632,0.0,0.0,0.0,0.052632,0.052632,0.052632,0.052632,0.0,0.0,0.0,0.000000,0.052632,0.0

In [59]:
k = 11

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)


kmeans = KMeans(n_clusters=k, random_state=42).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([ 9, 10,  6,  2,  1,  5,  0,  7,  3,  8], dtype=int32)

In [61]:
toronto_merged = toronto_grouped[["Borough","Cluster Labels"]]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = df_2.join(toronto_merged.set_index('Borough'), on='Borough')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,9
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,9
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,9
3,M1G,Scarborough,Woburn,43.770992,-79.216917,9
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,9


In [66]:
#create map
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
toronto_merged['Cluster Labels'].fillna(0, inplace=True)
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
#     print(cluster)
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters